# 知识工程-作业2

2024214500 叶璨铭


## 代码与文档格式说明

本文档使用Jupyter Notebook编写，所以同时包括了实验文档和实验代码。

本次实验项目采用了类似于 Quarto + nbdev 的方法来同步Jupyter Notebook代码到python文件, 因而我们的实验文档导出为pdf和html格式可以进行阅读，而我们的代码也导出为python模块形式，可以作为代码库被其他项目使用。
我们这样做的好处是，避免单独管理一堆 .py 文件，防止代码冗余和同步混乱，py文件和pdf文件都是从.ipynb文件导出的，可以保证实验文档和代码的一致性。

本文档理论上支持多个格式，包括ipynb, html, docx, pdf, md 等，但是由于 quarto和nbdev 系统的一些bug，我们目前暂时只支持ipynb文件，以后有空的时候解决bug可以构建一个[在线文档网站](https://thu-coursework-machine-learning-for-big-data-docs.vercel.app/)。您在阅读本文档时，可以选择您喜欢的格式来进行阅读，建议您使用 Visual Studio Code (或者其他支持jupyter notebook的IDE, 但是VSCode阅读体验最佳) 打开 `ipynb`格式的文档来进行阅读。


为了记录我们自己修改了哪些地方，使用git进行版本控制，这样可以清晰地看出我们基于助教的代码在哪些位置进行了修改，有些修改是实现了要求的作业功能，而有些代码是对助教的代码进行了重构和优化。我将我在知识工程课程的代码，在作业截止DDL之后，开源到 https://github.com/2catycm/THU-Coursework-Knowledge-Engineering.git ，方便各位同学一起学习讨论。


## 实验准备

首先进行 环境配置和`nervaluate`库的安装
```bash
conda activate yuequ # 知识工程课程使用我最近科研的环境 “约取”，这样不需要重新安装torch等库
pip install uv # uv的pip速度比 pip 快，不过配置清华源的方式不同。
uv pip install nervaluate
```

![alt text](image.png)

为了让老师给的代码更加规范，提高可读性，先用ruff进行格式化。

```bash
uv pip install ruff
ruff format
```
![alt text](image-1.png)

注意这个压缩包有MACOS搞出来的无用文件，我们直接删掉

```bash
rm -rf __MACOSX
```

结合 DRY 软件工程原则，我们参考fastai的文档规范来对函数做注释，防止重复声明很多次类型（助教使用的doc string 风格，会重复声明很多次，不好维护）


## 下载数据放到`./data/raw`文件夹下

注意到助教给出的压缩包已经下载了 MSRANER 数据集。

从 github 上可以看到，这个数据集训练集(46,364)，测试集(4,365)，github上还有一个 link.txt和test1.txt，以及一个转换train到pkl的python脚本。



## 实现 BIO 和 BIOES 标注模式
> 使用BIO和BIOES标注模式分别对train1.txt和testright1.txt中的标签进行预处理，得到每个字对应的标签。

根据文档要求，我们看到 preprocess.py 的 process 函数，`def process(file, target, mode="bio"):` ，输入一个文件路径，把处理好的东西放到输出文件，需要支持bio和bioes两个模式。
下面的main处理了train1.txt和testright1.txt。具体处理的逻辑是，
- 文件的每一行是一个句子或者说一个样本
- 每一行用空格分开可以得到单词列表
- 每个单词带有标注，用/分开
- bio和bioes函数需要处理单个单词的 chars与tag。


我们需要实现上面的`def bio(chars, tag):`, chars是单词和标签交替出现的句子


现在我们来复习一下课件，然而课件上虽然讲了条件随机场之类的，并没有详细讲解BIO和BIOES。所以我自己查阅一下资料。

根据论文《基于深度学习与规则结合的海运货物邮件命名实体识别》，BIO 全称是 B-begin, I-inside, O-outside，BIOES是 B-begin, I-inside, O-outside, E-end, S-single，属于是最常用的序列标注模式。其中这两个都有BIO，含义是一样的，BIOES多ES。具体含义是这样的：

- B（Begin） ：表示实体的开始位置。
- I（Inside） ：表示实体的中间位置。
- O（Outside） ：表示不属于任何实体的部分。
- E（End） ：表示实体的结束位置。
- S（Single） ：表示单个字符且本身是一个独立的实体。


在刚才那篇论文中采用了BIOES，因为BIOES相对于BIO更加细致，可以用于“多实体并列”。

> 追问：什么是“多实体并列”？什么是嵌套实体？什么是复杂实体？
> 
> 1. 一个实体的文本范围完全包含在另一个实体的文本范围内，形成“实体套实体”的结构。
> “他在北京大学读书。”（“北京”是地点实体，“北京大学”是组织机构实体，前者嵌套在后者内部）
> 



回顾老师课上讲的，NER任务包括定位和分类两步，所以实际的标注中，BIOES代表了位置，还要再加一个减号，然后标记上实体类别，比如刚才那个论文有5个类别。比如 




现在我们看回数据集的tag是什么意思，我们需要阅读微软原本的文档，否则不知道怎么处理。需要参考 https://faculty.washington.edu/levow/papers/sighan06.pdf， 但是这篇论文忽略了很多细节，论文中说的类别是(PER)SON, (LOC)ATION, (ORG)ANIZATION, or (G)EO-(P)OLITICAL，(E)NTITY，但是他给的数据集实际上是这样的：
- o，对应BIO的O
- nt/ns/nr 等，代表类别。

于是我们只好查询其他的资料，搞清楚nt ns nr是什么东西，因为课件上也没有提到。

根据PaddleNLP库的解释，常用的标签有这些
![alt text](image-2.png)





现在我大概搞懂了我们要干什么。
如果给我的类别是o，那自然就是O；否则要区分是不是单个词组成实体的情况，还要区分一下是不是新的实体开始了。
如果类别变换了，那应该也不是一个实体。

这是我大概的总结，但是我还是没找到权威的资料，告诉我规范的BIO和BIOES的权威规则。
为此，我现在要重新搜索一下代码而不是论文。AI搜索给了我相关的库是HanLP和PaddleNLP，我现在要看看这两个库是怎么处理的。

然而我看了文档找了半天没找到BIO和BIOES的处理函数。

倒是发现 HanLP的演示很炫酷 https://hanlp.hankcs.com/demos/ner.html

这里指出了三种标注规范，我们的数据表面上是MSRA的数据集，应该类似https://hanlp.hankcs.com/docs/annotations/ner/msra.html才对，但是我自己看其实标注格式类似于PKU规范 https://hanlp.hankcs.com/docs/annotations/pos/pku.html

而HanLP没有管BIO和BIOES的问题。

既然没有统一标准，我就按照我看到的这些资料自己来理解实现。这里面有几个我觉得很重要但是已有资料讲得不清楚的点
1. BIO和BIOES到底是对单个中文字符标注的，还是对单个单词标注的？
- 老师课件上前后提到了两种模式，一开始课件说NER的应用场景之一是可以解决分词问题的未登录词，所以可以改进分词效果，这个表述是说先搞NER，对每个单字标注BIO，然后自然自动地也把单词分开了。
- 然而课件后面随即有提到“因此命名实体识别的第一步就是确定词边界，即分词；”，那这个就是词已经分好了不需要我们分，只需要预测每个单词的标签，多个单词组合成一个实体。老师课件中的例子确实是这样。
- 但是回到我们的MSRA数据集，词是分好了，按照道理来说应该属于第二个模式，但是仔细看这个数据集分词的粒度，一个大的实体是合并在一起的，而没有按照分词去分开，比如“副总理兼外长/o” 本来是 “副总理/o 兼/o 外长/o”，这样的话，我们就需要按照第一个模式来处理，即对每个字进行标注。
- 看回HANLP，输出的格式是字符串的开始位置和结束位置构成的字典。

2. S是单字还是单词？函数是否应该有状态？
- 确定了问题1后，那很明显，S是单字，我们只需要考虑char，函数没有状态。
3. 嵌套问题无法考虑？

现在可以安心地开始写代码了。

In [ ]:
#| export
from typing import List

def bio(
    chars: str, tag: str
) -> List[List[str]]:  # Output: each sub list is a pair of character and tag
    """
    BIO tagging format
    Examples:
        Input: "北京大学", "nt"
        Output: [["北", "B-NT"], ["京", "I-NT"], ["大", "I-NT"], ["学", "I-NT"]]
    """
    tag = tag.upper()  # 根据助教的实例，输出的格式要求大写
    if tag == "O":
        return [[char, tag] for char in chars]  # 不是实体，返回一个O
    else:
        return [[chars[0], f"B-{tag}"]] + [
            [char, f"I-{tag}"] for char in chars[1:] # 如果是单个字的情况下，那就是 B-tag
        ]


def bioes(
    chars: str, tag: str
) -> List[List[str]]:  # Output: each sub list is a pair of character and tag
    """
    BIOES tagging format
    Examples:
        Input: "北京大学", "nt"
        Output: [["北", "B-NT"], ["京", "I-NT"], ["大", "I-NT"], ["学", "E-NT"]]
    """
    tag = tag.upper()
    if tag == "O":
        return [[char, tag] for char in chars]
    elif len(chars) == 1:
        return [[chars[0], f"S-{tag}"]] # 单个字的实体
    else:
        return [[chars[0], f"B-{tag}"]] + [
            [char, f"I-{tag}"] for char in chars[1:-1] # 如果是两个字的情况下, chars[1:-1]为空
        ] + [[chars[-1], f"E-{tag}"]]

In [ ]:
# 测试一下
from fastcore.test import test_eq
# 例子
test_eq(bio("北京大学", "nt"), 
        [['北', 'B-NT'], ['京', 'I-NT'], 
         ['大', 'I-NT'], ['学', 'I-NT']])
test_eq(bioes("北京大学", "nt"),
        [['北', 'B-NT'], ['京', 'I-NT'], 
         ['大', 'I-NT'], ['学', 'E-NT']])
# 非实体
test_eq(bio("北", "O"), [['北', 'O']])
test_eq(bioes("北", "O"), [['北', 'O']])
# 单个字的实体
test_eq(bio("北", "nt"), [['北', 'B-NT']])
test_eq(bioes("北", "nt"), [['北', 'S-NT']])
# 两个字的实体
test_eq(bio("北京", "nt"), [['北', 'B-NT'], ['京', 'I-NT']])
test_eq(bioes("北京", "nt"), [['北', 'B-NT'], ['京', 'E-NT']])